In [1]:
# From: https://github.com/hunkim/DeepLearningZeroToAll/
# Code: lab-12-4-rnn_long_char.py
# 
# Refactored into notebook

# import

In [2]:
from __future__ import print_function
import tensorflow as tf
import numpy as np
from tensorflow.contrib import rnn
import pprint
pp = pprint.PrettyPrinter(indent=4)

In [3]:
tf.set_random_seed(777)  # reproducibility

# training sentence

In [4]:
sentence = ("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.")

# making dataset

In [23]:
char_set = list(set(sentence))
char_dic = {w: i for i, w in enumerate(char_set)}

data_dim = len(char_set)
hidden_size = len(char_set)
num_classes = len(char_set)
sequence_length = 10  # Any arbitrary number
learning_rate = 0.1

print("data_dim: %s" % data_dim)
print("hidden_size: %s" % hidden_size)
print("num_classes: %s" % num_classes)
print("sequence_length: %s" % sequence_length)

dataX = []
dataY = []
for i in range(0, len(sentence) - sequence_length):
    x_str = sentence[i:i + sequence_length]
    y_str = sentence[i + 1: i + sequence_length + 1]
    print(i, x_str, '->', y_str)

    x = [char_dic[c] for c in x_str]  # x str to index
    y = [char_dic[c] for c in y_str]  # y str to index

    dataX.append(x)
    dataY.append(y)

batch_size = len(dataX)
print("batch_size: %s" % batch_size)
pp.pprint(dataX[0])
pp.pprint(dataY[0])

data_dim: 25
hidden_size: 25
num_classes: 25
sequence_length: 10
0 if you wan -> f you want
1 f you want ->  you want 
2  you want  -> you want t
3 you want t -> ou want to
4 ou want to -> u want to 
5 u want to  ->  want to b
6  want to b -> want to bu
7 want to bu -> ant to bui
8 ant to bui -> nt to buil
9 nt to buil -> t to build
10 t to build ->  to build 
11  to build  -> to build a
12 to build a -> o build a 
13 o build a  ->  build a s
14  build a s -> build a sh
15 build a sh -> uild a shi
16 uild a shi -> ild a ship
17 ild a ship -> ld a ship,
18 ld a ship, -> d a ship, 
19 d a ship,  ->  a ship, d
20  a ship, d -> a ship, do
21 a ship, do ->  ship, don
22  ship, don -> ship, don'
23 ship, don' -> hip, don't
24 hip, don't -> ip, don't 
25 ip, don't  -> p, don't d
26 p, don't d -> , don't dr
27 , don't dr ->  don't dru
28  don't dru -> don't drum
29 don't drum -> on't drum 
30 on't drum  -> n't drum u
31 n't drum u -> 't drum up
32 't drum up -> t drum up 
33 t drum up  ->  dru

# Stacked RNN

In [24]:
X = tf.placeholder(tf.int32, [None, sequence_length])
Y = tf.placeholder(tf.int32, [None, sequence_length])

# One-hot encoding
X_one_hot = tf.one_hot(X, num_classes)
print(X_one_hot)  # check out the shape


# Make a lstm cell with hidden_size (each unit output vector size)
def lstm_cell():
    cell = rnn.BasicRNNCell(hidden_size)
    # chane to lstm when training
    #cell = rnn.BasicLSTMCell(hidden_size, state_is_tuple=True)
    return cell

multi_cells = rnn.MultiRNNCell([lstm_cell() for _ in range(2)], state_is_tuple=True)

# outputs: unfolding size x hidden size, state = hidden size
outputs, _states = tf.nn.dynamic_rnn(multi_cells, X_one_hot, dtype=tf.float32)

print(outputs)
print(_states)

Tensor("one_hot_7:0", shape=(?, 10, 25), dtype=float32)
Tensor("rnn_1/transpose:0", shape=(?, 10, 25), dtype=float32)
(<tf.Tensor 'rnn_1/while/Exit_2:0' shape=(?, 25) dtype=float32>, <tf.Tensor 'rnn_1/while/Exit_3:0' shape=(?, 25) dtype=float32>)


In [25]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

results, hidden_states = sess.run([outputs, _states], feed_dict={X: dataX, Y: dataY})

print(np.shape(results))
print(np.shape(hidden_states))

(170, 10, 25)
(2, 170, 25)


# Softmax

In [26]:
# FC layer
X_for_fc = tf.reshape(outputs, [-1, hidden_size])
print(X_for_fc)

outputs = tf.contrib.layers.fully_connected(X_for_fc, num_classes, activation_fn=None)
print(outputs)

# reshape out for sequence_loss
outputs = tf.reshape(outputs, [batch_size, sequence_length, num_classes])
print(outputs)

Tensor("Reshape_2:0", shape=(?, 25), dtype=float32)
Tensor("fully_connected_1/BiasAdd:0", shape=(?, 25), dtype=float32)
Tensor("Reshape_3:0", shape=(170, 10, 25), dtype=float32)


In [27]:
# All weights are 1 (equal weights)
weights = tf.ones([batch_size, sequence_length])

sequence_loss = tf.contrib.seq2seq.sequence_loss(
    logits=outputs, targets=Y, weights=weights)
mean_loss = tf.reduce_mean(sequence_loss)
train_op = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(mean_loss)

# Training and print results

In [20]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for i in range(500):
    _, l, results = sess.run(
        [train_op, mean_loss, outputs], feed_dict={X: dataX, Y: dataY})
    for j, result in enumerate(results):
        index = np.argmax(result, axis=1)
        print(i, j, ''.join([char_set[t] for t in index]), l)

0 0 hgggauuuuu 3.21983
0 1 w......ggg 3.21983
0 2 ...fffffgm 3.21983
0 3 aaauuuuuu' 3.21983
0 4 ccuuuuuu'' 3.21983
0 5 uuugggg''' 3.21983
0 6 ..ggge'''' 3.21983
0 7 hggee''''' 3.21983
0 8 glee''''.u 3.21983
0 9 km..''...u 3.21983
0 10 ''''''..uu 3.21983
0 11 .'''...uuu 3.21983
0 12 ''''.uuuuu 3.21983
0 13 ccnnfuuuuu 3.21983
0 14 ...fuuuug' 3.21983
0 15 nnfuuuug'' 3.21983
0 16 uuuuuu'''' 3.21983
0 17 huu''''''' 3.21983
0 18 u''''''''' 3.21983
0 19 aaagg      3.21983
0 20 .ggg       3.21983
0 21 ggguuu''   3.21983
0 22 .uuuffffff 3.21983
0 23 ruuuuummfk 3.21983
0 24 hhkkfffkkm 3.21983
0 25 hmttttkkmm 3.21983
0 26 fffffkkkmm 3.21983
0 27 o ookkkmmm 3.21983
0 28 ..ckkmmmmy 3.21983
0 29 ackkmmmmyb 3.21983
0 30 ckk,,mmbbb 3.21983
0 31 kk,....w.u 3.21983
0 32 ym....w... 3.21983
0 33 '''''eeuuf 3.21983
0 34 ..thhtufff 3.21983
0 35 ahhhouffff 3.21983
0 36 ehweuuffff 3.21983
0 37 uuuuufffff 3.21983
0 38 oouuffffff 3.21983
0 39 .uffffffff 3.21983
0 40 uuffffffff 3.21983
0 41 ffffffffff 3.21983
0 

 o          2.8707
3 75            2.8707
3 76            2.8707
3 77            2.8707
3 78 o          2.8707
3 79            2.8707
3 80            2.8707
3 81            2.8707
3 82            2.8707
3 83            2.8707
3 84 o          2.8707
3 85            2.8707
3 86            2.8707
3 87            2.8707
3 88            2.8707
3 89            2.8707
3 90            2.8707
3 91 o          2.8707
3 92            2.8707
3 93            2.8707
3 94            2.8707
3 95            2.8707
3 96 o          2.8707
3 97            2.8707
3 98            2.8707
3 99            2.8707
3 100            2.8707
3 101            2.8707
3 102 o          2.8707
3 103            2.8707
3 104            2.8707
3 105            2.8707
3 106 o          2.8707
3 107 o          2.8707
3 108            2.8707
3 109            2.8707
3 110            2.8707
3 111            2.8707
3 112 o          2.8707
3 113            2.8707
3 114            2.8707
3 115            2.8707
3 116 o          2.870

159            2.83723
6 160            2.83723
6 161            2.83723
6 162            2.83723
6 163            2.83723
6 164            2.83723
6 165            2.83723
6 166            2.83723
6 167            2.83723
6 168            2.83723
6 169            2.83723
7 0            2.82169
7 1            2.82169
7 2            2.82169
7 3            2.82169
7 4            2.82169
7 5            2.82169
7 6            2.82169
7 7            2.82169
7 8            2.82169
7 9            2.82169
7 10            2.82169
7 11            2.82169
7 12            2.82169
7 13            2.82169
7 14            2.82169
7 15            2.82169
7 16            2.82169
7 17            2.82169
7 18            2.82169
7 19            2.82169
7 20            2.82169
7 21            2.82169
7 22            2.82169
7 23            2.82169
7 24            2.82169
7 25            2.82169
7 26            2.82169
7 27            2.82169
7 28            2.82169
7 29            2.82169
7 30            2

           2.75521
10 18            2.75521
10 19            2.75521
10 20            2.75521
10 21            2.75521
10 22            2.75521
10 23            2.75521
10 24            2.75521
10 25            2.75521
10 26            2.75521
10 27            2.75521
10 28            2.75521
10 29            2.75521
10 30            2.75521
10 31            2.75521
10 32            2.75521
10 33            2.75521
10 34            2.75521
10 35            2.75521
10 36            2.75521
10 37            2.75521
10 38            2.75521
10 39            2.75521
10 40            2.75521
10 41            2.75521
10 42            2.75521
10 43            2.75521
10 44            2.75521
10 45            2.75521
10 46            2.75521
10 47            2.75521
10 48            2.75521
10 49            2.75521
10 50            2.75521
10 51            2.75521
10 52            2.75521
10 53            2.75521
10 54            2.75521
10 55            2.75521
10 56            2.75521
10 57 

     ee     2.58848
13 120    ee      2.58848
13 121   to     e 2.58848
13 122  oo    ee  2.58848
13 123  o     e   2.58848
13 124       e    2.58848
13 125      e     2.58848
13 126    ee      2.58848
13 127   eo       2.58848
13 128   o        2.58848
13 129  o         2.58848
13 130            2.58848
13 131            2.58848
13 132            2.58848
13 133   o o      2.58848
13 134  o oo      2.58848
13 135   oo       2.58848
13 136            2.58848
13 137          o 2.58848
13 138      e  oo 2.58848
13 139    to  oo  2.58848
13 140   te  oo   2.58848
13 141  ee eoo    2.58848
13 142  o oooo    2.58848
13 143    ooo     2.58848
13 144   ooo      2.58848
13 145  ooe       2.58848
13 146  o         2.58848
13 147            2.58848
13 148            2.58848
13 149            2.58848
13 150            2.58848
13 151            2.58848
13 152            2.58848
13 153            2.58848
13 154            2.58848
13 155            2.58848
13 156            2.58848
13 157            

...

484 88 pn them ta 0.229101
484 89 e them tas 0.229101
484 90 dthem task 0.229101
484 91 toem tosks 0.229101
484 92 her tosks  0.229101
484 93 em tosks a 0.229101
484 94 r tosks an 0.229101
484 95  tosks and 0.229101
484 96 tosks and  0.229101
484 97 hsks and w 0.229101
484 98 nks and wo 0.229101
484 99  s and wor 0.229101
484 100 , and work 0.229101
484 101  and work, 0.229101
484 102 tnd work,  0.229101
484 103 nd dork, b 0.229101
484 104 d work, bu 0.229101
484 105  aork, but 0.229101
484 106 took, but  0.229101
484 107 ook, but r 0.229101
484 108   , but ra 0.229101
484 109  , but rat 0.229101
484 110 , but rath 0.229101
484 111  dut rathe 0.229101
484 112 tui rather 0.229101
484 113 ut rather  0.229101
484 114 p rather t 0.229101
484 115 hrather te 0.229101
484 116 tather tea 0.229101
484 117  ther teac 0.229101
484 118 nher teach 0.229101
484 119 her toach  0.229101
484 120 em toach t 0.229101
484 121 r toach th 0.229101
484 122  toach the 0.229101
484 123 toach them 0.229101
484 

158 tmmensity  0.229065
487 159 pmensity o 0.229065
487 160  ensity of 0.229065
487 161  nsity of  0.229065
487 162 msity of t 0.229065
487 163 dity of th 0.229065
487 164 sty of the 0.229065
487 165 py of the  0.229065
487 166   of the s 0.229065
487 167  of the se 0.229065
487 168 tf the sea 0.229065
487 169   the sea. 0.229065
488 0 p you want 0.229047
488 1 oyou want  0.229047
488 2 tou want t 0.229047
488 3 ou want to 0.229047
488 4   want to  0.229047
488 5  want to b 0.229047
488 6 tont to bu 0.229047
488 7 ont to bui 0.229047
488 8 nd to buil 0.229047
488 9 d to build 0.229047
488 10 hdo build  0.229047
488 11 to luild a 0.229047
488 12 h luild a  0.229047
488 13  luild a s 0.229047
488 14 tuild a sh 0.229047
488 15 utld a shi 0.229047
488 16  ld a ship 0.229047
488 17 pd a ship, 0.229047
488 18 e a ship,  0.229047
488 19  anship, d 0.229047
488 20 tnship, do 0.229047
488 21 nship, don 0.229047
488 22 thip, don' 0.229047
488 23  ip, don't 0.229047
488 24 ep, don't  0.229047
488

34 toum up pe 0.229017
491 35  um up peo 0.229017
491 36  m up peop 0.229017
491 37 p up peopl 0.229017
491 38  tp people 0.229017
491 39 tp people  0.229017
491 40 p people t 0.229017
491 41 epeople to 0.229017
491 42 teople tog 0.229017
491 43 eople toge 0.229017
491 44 rple toget 0.229017
491 45 nle togeth 0.229017
491 46 ee togethe 0.229017
491 47 estogether 0.229017
491 48 reogether  0.229017
491 49 to ether t 0.229017
491 50 h ether to 0.229017
491 51 nether to  0.229017
491 52 ether to c 0.229017
491 53 rher to co 0.229017
491 54 hem to col 0.229017
491 55 er to coll 0.229017
491 56 r te colle 0.229017
491 57  te collec 0.229017
491 58 to bollect 0.229017
491 59 h bollect  0.229017
491 60 nbollect w 0.229017
491 61 tollect wo 0.229017
491 62 hllect woo 0.229017
491 63 nlect wood 0.229017
491 64 eect wood  0.229017
491 65 est wood a 0.229017
491 66 rt wood an 0.229017
491 67 h wood and 0.229017
491 68 hrood and  0.229017
491 69 tord and d 0.229017
491 70 ord and do 0.229017
491 7

494 128 ethem to l 0.229013
494 129 toem ta lo 0.229013
494 130  em ta lon 0.229013
494 131 em ta long 0.229013
494 132   ta long  0.229013
494 133  ta long f 0.229013
494 134 to bong fo 0.229013
494 135   bong for 0.229013
494 136 nbong for  0.229013
494 137 tong for t 0.229013
494 138 eng for th 0.229013
494 139 n' for the 0.229013
494 140 d for the  0.229013
494 141  for the e 0.229013
494 142 tor the en 0.229013
494 143  r the end 0.229013
494 144 n the endl 0.229013
494 145  the endle 0.229013
494 146 toemendles 0.229013
494 147  emendless 0.229013
494 148 emendless  0.229013
494 149  tndless i 0.229013
494 150 tndless im 0.229013
494 151  dless imm 0.229013
494 152 d ess imme 0.229013
494 153  ess immen 0.229013
494 154 ess immens 0.229013
494 155  s immensi 0.229013
494 156 s immensit 0.229013
494 157 simmensity 0.229013
494 158 tmmensity  0.229013
494 159 pmensity o 0.229013
494 160  ensity of 0.229013
494 161  nsity of  0.229013
494 162  dity of t 0.229013
494 163 dity of th 0

498 16 pld a ship 0.229032
498 17 pd a ship, 0.229032
498 18 e a ship,  0.229032
498 19  a ship, d 0.229032
498 20 tnship, do 0.229032
498 21 nship, don 0.229032
498 22 thip, don' 0.229032
498 23 sip, don't 0.229032
498 24 ep, don't  0.229032
498 25 p, don't d 0.229032
498 26 e don't dr 0.229032
498 27  bon't dru 0.229032
498 28 ton't arum 0.229032
498 29  n't drum  0.229032
498 30 n't drum u 0.229032
498 31 dt drum up 0.229032
498 32 t drum up  0.229032
498 33  trum up p 0.229032
498 34 toum up pe 0.229032
498 35  um up peo 0.229032
498 36  m up peop 0.229032
498 37 p up peopl 0.229032
498 38  tp people 0.229032
498 39 tp people  0.229032
498 40 p people t 0.229032
498 41 epeople to 0.229032
498 42 teople tog 0.229032
498 43 eople toge 0.229032
498 44  ple toget 0.229032
498 45 nle togeth 0.229032
498 46 ee togethe 0.229032
498 47 estogether 0.229032
498 48  together  0.229032
498 49 to ether t 0.229032
498 50   ether to 0.229032
498 51 nether to  0.229032
498 52  ther to c 0.229032
4

In [21]:
# Let's print the last char of each result to check it works
results = sess.run(outputs, feed_dict={X: dataX})
for j, result in enumerate(results):
    index = np.argmax(result, axis=1)
    if j is 0:  # print all for the first result to make a sentence
        print(''.join([char_set[t] for t in index]), end='')
    else:
        print(char_set[index[-1]], end='')

p you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.